In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense,Input,Bidirectional,LSTM,GRU,SimpleRNN,Dropout,LayerNormalization , Concatenate,Conv2D,MaxPooling2D,Layer,BatchNormalization,GlobalMaxPooling1D,Conv1D,Activation, Dot
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from sklearn.utils import class_weight
import librosa
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
import pickle
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from tensorflow.keras.models import clone_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.utils import plot_model

2024-07-04 12:37:16.187724: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 12:37:16.187858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 12:37:16.299805: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
file_path = '/content/drive/MyDrive/padded_text_combined_augmented.pickle'

# Open the file in binary read mode
with open(file_path, 'rb') as f:
    # Load the data from the file
    text = pickle.load(f)

file_path = '/content/drive/MyDrive/labels_combined_augmented.pickle'

# Open the file in binary read mode
with open(file_path, 'rb') as f:
    # Load the data from the file
    labels = pickle.load(f)


In [8]:
early_stopping = EarlyStopping(
    monitor='val_loss',      # Metric to monitor
    patience=5,              # Number of epochs with no improvement after which training will be stopped
    verbose=1,               # Verbosity mode
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',     # Metric to monitor
    factor=0.5,             # Factor by which the learning rate will be reduced
    patience=3,             # Number of epochs with no improvement after which learning rate will be reduced
    verbose=1,              # Verbosity mode
    min_lr=1e-7             # Lower bound on the learning rate
)

In [9]:
vocab_size = 2843

In [10]:
X_text = np.array(text)
y = np.array(labels)

In [11]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [12]:
input_layer = Input(shape=(98,), dtype='int32')

# Embedding layer to convert integer-encoded input to dense vectors of fixed size
embedding_dim = 200
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=98)(input_layer)
# BiLSTM branch
bilstm_model2 = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
bilstm_pool = GlobalMaxPooling1D()(bilstm_model2)
# CNN branch
cnn = Conv1D(filters=128, kernel_size=5, activation='relu')(embedding_layer)
cnn_pool = GlobalMaxPooling1D()(cnn)
# Concatenate the outputs of the two branches
concatenated = Concatenate()([bilstm_pool, cnn_pool])


dense1 = Dense(64,activation='relu')(concatenated)
output_layer = Dense(5, activation='softmax')(dense1)
model_text = Model(inputs=input_layer,outputs=output_layer)

model_text.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the summary of the combined model
model_text.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 98)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 98, 200)   │    568,600 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 98, 128)   │    135,680 │ embedding[0][0]   │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 94, 128)   │    128,128 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ bidirectional[0]… │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d[0][0]      │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │     16,448 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 5)         │        325 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 849,181 (3.24 MB)

 Trainable params: 849,181 (3.24 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
val_accuracies = []
train_accuracies_per_epoch = []
val_accuracies_per_epoch = []
train_losses_per_epoch = []
val_losses_per_epoch = []

for train_index, val_index in kf.split(X_text):

    X_text_train, X_text_val = X_text[train_index], X_text[val_index]
    y_train, y_val = y[train_index], y[val_index]

    history = model_text.fit(
        X_text_train,  # Input data
        y_train,                       # Labels
        batch_size=32,                 # Batch size
        epochs=30,                     # Number of epochs
        validation_data=(X_text_val, y_val),
        callbacks=[reduce_lr,early_stopping]

    )
    val_accuracy = history.history['val_accuracy'][-1]
    val_accuracies.append(val_accuracy)
    print(f"Fold validation accuracy: {val_accuracy}")

    # Store the accuracies
    train_accuracies_per_epoch.append(history.history['accuracy'])
    val_accuracies_per_epoch.append(history.history['val_accuracy'])
    train_losses_per_epoch.append(history.history['loss'])
    val_losses_per_epoch.append(history.history['val_loss'])


average_val_accuracy = np.mean(val_accuracies)
print("Average Validation Accuracy:", average_val_accuracy)




Epoch 1/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.2606 - loss: 1.5785 - val_accuracy: 0.5013 - val_loss: 1.2367 - learning_rate: 0.0010
Epoch 2/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5748 - loss: 1.0743 - val_accuracy: 0.5768 - val_loss: 1.0842 - learning_rate: 0.0010
Epoch 3/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.7453 - loss: 0.7047 - val_accuracy: 0.6060 - val_loss: 1.0492 - learning_rate: 0.0010
Epoch 4/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8500 - loss: 0.4453 - val_accuracy: 0.5892 - val_loss: 1.1170 - learning_rate: 0.0010
Epoch 5/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8673 - loss: 0.3716 - val_accuracy: 0.5634 - val_loss: 1.2675 - learning_rate: 0.0010
Epoch 6/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8944 - loss: 0.3095
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8944 - loss: 0.3